%%markdown

# GigaSpaces API Examples
This notebook will demonstrate the basics of the GigaSpaces API.



%%markdown

##### POJO definition

1.  The accompanying Maven project can be found at ~/gs-core-dev-training/jupyter. `mvn install` has been run and it is now accessible to this notebook.

In [ ]:
%%pom
repositories:
  - id: org.openspaces
    url: https://maven-repository.openspaces.org
dependencies:
  - groupId: org.gigaspaces
    artifactId: xap-openspaces
    version: 16.4.0
    
  - groupId: com.gigaspaces
    artifactId: xap-jdbc
    version: 16.4.0
    
  - groupId: com.gigaspaces.training
    artifactId: model
    version: 1.0
        

%%markdown

##### Here is a look at the pojo that will be used in this lesson.

In [ ]:
%%bash

cat jupyter-maven/model/src/main/java/com/gigaspaces/training/model/MyPojo.java

In [ ]:
%%classpath

In [ ]:
%%java

import com.gigaspaces.training.model.MyPojo;

// readById and readByIds
import com.gigaspaces.client.ReadByIdsResult;
import com.gigaspaces.query.IdQuery;
import com.gigaspaces.query.IdsQuery;

// clear
import com.gigaspaces.client.ClearModifiers;

// SpaceDocuments
import com.gigaspaces.document.DocumentProperties;
import com.gigaspaces.document.SpaceDocument;
import com.gigaspaces.metadata.SpaceTypeDescriptor;
import com.gigaspaces.metadata.SpaceTypeDescriptorBuilder;
import java.util.Map;
import java.util.HashMap;

import com.j_spaces.core.client.SQLQuery;

// jdbc
import java.sql.Connection;
import java.sql.DriverManager;
import java.sql.ResultSet;
import java.sql.Statement;
import java.sql.SQLException;
import java.util.Properties;

import org.openspaces.core.GigaSpace;
import org.openspaces.core.GigaSpaceConfigurer;
import org.openspaces.core.space.EmbeddedSpaceConfigurer;



In [ ]:
%%java
// an example of instantiating an embedded space.

EmbeddedSpaceConfigurer configurer = new EmbeddedSpaceConfigurer("demo");
GigaSpace gigaSpace = new GigaSpaceConfigurer(configurer).gigaSpace();


In [ ]:
%%java

// write objects
    
for( int i=0; i < 10; i++) {
    MyPojo myPojo = new MyPojo();
    myPojo.setId(i);
    myPojo.setMessage("msg: " + i);
    gigaSpace.write(myPojo);
}

In [ ]:
%%java
// count

int numObjects = gigaSpace.count(new MyPojo());
System.out.println("The number of MyPojo objects is: " + numObjects);

%%markdown

##### readById

readById is used with an idQuery and is used to read based on id. It is a very efficient API when compared to the other read APIs.


In [ ]:
%%java

// readById example
Integer id = 3;
IdQuery<MyPojo> idQuery = new IdQuery<MyPojo>(MyPojo.class, id);

MyPojo returnValue = gigaSpace.readById(idQuery);

System.out.println("Value returned: " + returnValue);


%%markdown

##### readByIds

Similiar to readById, readByIds is used to read based on ids. The collection of objects returned can be read from the ReadByIdsResult object.


In [ ]:
%%java

// readByIds example
Integer[] ids = new Integer[]{1, 3, 5};
IdsQuery<MyPojo> idsQuery = new IdsQuery<MyPojo>(MyPojo.class, ids);

ReadByIdsResult<MyPojo> results = gigaSpace.readByIds(idsQuery);

for (MyPojo value: results) {
    System.out.println("Value returned: " + value);
}

%%markdown

##### template matching is an efficient way to query for objects

1. It should be used for `=` matching.
2. The template should be the type of the object you wish to query for.
3. All null values in the template object are treated as wild card.
4. GigaSpaces will match the non-null values with the objects in the space. 

In [ ]:
%%java

// read example using template object
MyPojo template = new MyPojo();
template.setId(7);

MyPojo returnValue = gigaSpace.read(template);

System.out.println("read returned MyPojo with id of: " + returnValue.getId());

In [ ]:
%%markdown

##### SQLQuery

1. GigaSpaces can query objects based on SQLQuery.

In [ ]:
%%java

// read example using SQLQuery
int id = 7;
SQLQuery<MyPojo> sqlQuery = new SQLQuery<MyPojo>(MyPojo.class, "id = ?");
sqlQuery.setParameter(1, id);

MyPojo sqlQueryReturnValue = gigaSpace.read(sqlQuery);

System.out.println("read returned MyPojo with id of: " + sqlQueryReturnValue.getId());

%%markdown

##### take
1. Take is like read, except it deletes the object from the space.

In [ ]:
%%java

// take example
template = new MyPojo();
template.setId(9);

MyPojo returnValue = gigaSpace.take(template);

System.out.println("take removed: " + returnValue);

%%markdown

##### clear
1. clear also deletes the object from the space but it doesn't not return the object.

In [ ]:
%%java

// clear example
template = new MyPojo();
template.setId(9);

int numItems = gigaSpace.clear(template, ClearModifiers.NONE);

System.out.println("clear removed this many items: " + numItems);

%%markdown

##### SpaceDocument example

1. Register type

In [ ]:
%%java

SpaceTypeDescriptor typeDescriptor = new SpaceTypeDescriptorBuilder("ProductDescription")
            .idProperty("id", false)
            .create();

// Register type:
gigaSpace.getTypeManager().registerTypeDescriptor(typeDescriptor);


%%markdown

##### Writing a SpaceDocument object
    
1. Write using DocumentProperties type

In [ ]:
%%java

DocumentProperties properties = new DocumentProperties()
    .setProperty("id", "1")
    .setProperty("Features", new DocumentProperties()
    .setProperty("Manufacturer", "Acme")
    .setProperty("RequiresAssembly", true)
    .setProperty("NumberOfParts", 42));

SpaceDocument document1 = new SpaceDocument("ProductDescription", properties);
gigaSpace.write(document1);


%%markdown

##### Writing a SpaceDocument object
    
1. Write using Map type

In [ ]:
%%java

Map<String, Object> map = new HashMap<String, Object>();

map.put("id", "4");

Map<String, Object> features = new HashMap<String, Object>();
features.put("Manufacturer", "Acme");
features.put("RequiresAssembly", false);
features.put("Weight", 100);
map.put("Features", features);
        
SpaceDocument document2 = new SpaceDocument("ProductDescription", map);
gigaSpace.write(document2);


%%markdown

##### Read a SpaceDocument

In [ ]:
%%java

IdQuery<SpaceDocument> query = new IdQuery<SpaceDocument>("ProductDescription", "1");

SpaceDocument document = gigaSpace.readById(query);

if (document != null) {
    System.out.println("id: " +((Object) document.getProperty("id")).toString());
}
else {
    System.out.println("No space document read.");
}

In [ ]:
%%java

try {

    Properties props = new Properties();
    props.put("com.gs.embeddedQP.enabled", "true");

    Connection conn = DriverManager.getConnection("jdbc:gigaspaces:v3://localhost/demo", props);

    Statement statement = conn.createStatement();
    String query = "select * from \"com.gigaspaces.training.model.MyPojo\"";
    ResultSet rs = statement.executeQuery(query);

    while (rs.next()) {
        int id = rs.getInt("id");
        String message = rs.getString("message");
        System.out.println("id: " + id + ", message: " + message);
    }
} catch (SQLException e) {
    e.printStackTrace();
} catch (Exception e) {
    e.printStackTrace();
}    
